In [1]:
import pulp as pl
import pandas as pd
import numpy as np
from pulp import *

df = pd.read_csv("C:/Users/pooja.bhati/Downloads/GApoc/IP/return_cons.csv")

DateTime_df = df.set_index("Datetime")

dfNew=DateTime_df['2012-01-01':'2019-05-23']
dfNew.to_csv("C:/Users/pooja.bhati/Downloads/GApoc/IP/df1wft.csv", index=False)
dfNew.reset_index(level=0, inplace=True)

####################################### Walk Forward Testing Split##########################
IN_Sample_Size = 40
Out_Sample_Size = 10

result = pd.DataFrame(columns=['IN_Sample_Start_Date', 'IN_Sample_End_Date','Out_Sample_Start_Date', 'Out_Sample_End_Date'])

In_Sample_Start_Date = [dfNew.iloc[x]['Datetime'] for x in range(0, len(dfNew), Out_Sample_Size)]
In_Sample_End_Date = [dfNew.iloc[x]['Datetime'] for x in range(IN_Sample_Size-1, len(dfNew), Out_Sample_Size)]
Out_Sample_Start_Date = [dfNew.iloc[x]['Datetime'] for x in range(IN_Sample_Size, len(dfNew), Out_Sample_Size)]
Out_Sample_End_Date = [dfNew.iloc[x]['Datetime'] for x in range((IN_Sample_Size+Out_Sample_Size)-1, len(dfNew), Out_Sample_Size)]

df_wft = pd.DataFrame(list(zip(In_Sample_Start_Date, In_Sample_End_Date, Out_Sample_Start_Date, Out_Sample_End_Date)), 
               columns =['In_Sample_Start_Date', 'In_Sample_End_Date', 'Out_Sample_Start_Date', 'Out_Sample_End_Date'])

df_wft.to_csv("C:/Users/pooja.bhati/Downloads/GApoc/IP/walkforward.csv", index=False)


In [2]:
class IntegerProgrammingWts:

    def __init__(self,df,df_wft):
        self.df=df
        self.df_wft=df_wft
        self.DateTime_df = self.df.set_index("Datetime")
    
    
    
    def TrainingForStratgy(self,startDate,EndDate):  
        dfNew=self.DateTime_df[startDate:EndDate]
        dfSort=self.DateTime_df[startDate:EndDate].sort_values(['SG Strategy'],ascending=False)
        X=dfSort.iloc[0,:-1]
        Y=dfSort.iloc[0,-1]
        
        problem = pl.LpProblem("Strategy Problem",pl.LpMaximize)
        
        df1_cum = 20000*np.exp(np.log1p(dfNew).cumsum())
        
        R_CumSum=[(df1_cum.iloc[len(dfNew)-1,i]-df1_cum.iloc[0,i]) for i in range(len(df1_cum.columns))]
        X_cum=R_CumSum[:-1]
        Y_cum=R_CumSum[-1:]
        
        
        #Define Interger variables for each strategy
        variables=[]
        for i in range(len(X_cum)):
            s = pl.LpVariable(f's{i+1}',0,5,pl.LpInteger)
            variables.append(s)
            
        # Create the objective function
        problem += pl.lpSum([variables[a]* X_cum[a] for a in range(len(X_cum))])
        
        
        # Create constraints        
            
        problem += variables[0] >= 0
        problem += variables[1] >= 0
        problem += variables[2] >= 0
        problem += variables[3] >= 0
        problem += variables[4] >= 0
        problem += variables[5] >= 0
        problem += variables[6] >= 0
        problem += variables[7] >= 0
        problem += variables[8] >= 0
        problem += variables[9] >= 0
        problem += variables[10] >= 0
        problem += variables[11] >= 0
        problem += variables[12] >= 0
        problem += variables[13] >= 0
        problem += variables[14] >= 0
        
        problem += variables[0] <= 5
        problem += variables[1] <= 5
        problem += variables[2] <= 5
        problem += variables[3] <= 5
        problem += variables[4] <= 5
        problem += variables[5] <= 5
        problem += variables[6] <= 5
        problem += variables[7] <= 5
        problem += variables[8] <= 5
        problem += variables[9] <= 5
        problem += variables[10] <= 5
        problem += variables[11] <= 5
        problem += variables[12] <= 5
        problem += variables[13] <= 5
        problem += variables[14] <= 5
                
        
       
        problem += pl.lpSum([-(variables[a]* X[a]) for a in range(len(X))]) >= (-Y) #C1
        problem += pl.lpSum([variables[a] for a in range(len(X_cum))]) == 15 #C2
        print(problem)
        # Run problem 
        problem.solve()
        
        #Getting Solution
        solution = []
        solve=[]
        for i, x in enumerate(variables):
              solution.append(x.value())
        solve.append(problem.objective.value())        
              
        return solution,solve  
    
    ####################### Get Cummulative List from Testing data############################
    def TestingCumList(self,startDate,EndDate):
        dfTest=self.DateTime_df[startDate:EndDate]
        dfTestCum = 20000*np.exp(np.log1p(dfTest).cumsum())
        RTestCumSum=[((dfTestCum.iloc[len(dfTest)-1,i]-dfTestCum.iloc[0,i])) for i in range(len(dfTest.columns)-1)]
        return RTestCumSum
    #################################Calling Training and Testing functions#####################################
    def IPWeightsClaculation(self):
        TrainingData=[]
        TestValue=[]
        ObjectiveList=[]
        for i in range(len(self.df_wft)):                              
            strategyList=self.TrainingForStratgy(self.df_wft.iloc[i,0],self.df_wft.iloc[i,1])
            TrainingData.append(strategyList[0])
            ObjectiveList.append(strategyList[1])
            CumList=self.TestingCumList(self.df_wft.iloc[i,2],self.df_wft.iloc[i,3])
            TestValue.append([sum(x * y for x, y in zip(strategyList[0], CumList))])
                
        #######################Creating and merging DataFrame#############################
        TrainingStrategy=pd.DataFrame(TrainingData, columns=df.columns[1:-1])
        TrainingObjective=pd.DataFrame(ObjectiveList,columns=['Training Objective'])
        TesingDf=pd.DataFrame(TestValue,columns=['Testing Result'])
           
        FinalResult=pd.concat([self.df_wft,TrainingObjective,TesingDf,TrainingStrategy],axis=1)
        return FinalResult

In [ ]:
object_1=IntegerProgrammingWts(df,df_wft)
z=object_1.IPWeightsClaculation()

z.to_csv("C:/Users/pooja.bhati/Downloads/GApoc/IP/IPStrategy.csv", index=False) 

Strategy Problem:
MAXIMIZE
-637.490882567472*s1 + 486.3961088311298*s10 + 3315.319691541692*s11 + 1049.317943336824*s12 + 644.2136187178658*s14 + 344.9268484336826*s15 + 102.59429003910918*s2 + 1124.5186765751823*s3 + -242.4598088119492*s4 + 162.97575946924553*s5 + -1543.9408779271616*s6 + -2547.1089775739783*s8 + 1905.754298304535*s9 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: - 0.0224764243067 s1 - 0.0198825213485 s15 - 0.0143610952653 s2
 + 0.0983052832988 s8 - 0.0586122800584 s9 >= -0.0539830599335

_C32: s1 + s10 + s11 + s12 + s13 + s14 + s15 + s2 + s3 + s4

Strategy Problem:
MAXIMIZE
-322.2775800711752*s1 + 2500.1211298997077*s11 + -3.5185869354863826*s12 + 74.52281359681001*s13 + -51.290903262401116*s14 + -145.7908288267463*s15 + 667.3860827792669*s2 + 4366.484022693101*s3 + 227.16797260814928*s4 + 3105.023443631315*s5 + 64880.716711220215*s6 + -979.741712044397*s7 + 6485.1039849143235*s8 + 3788.718879817061*s9 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: 0.00936671169261 s1 - 0.0613647520864 s11 + 0.00885425889853 s15
 - 0.0297183327702 s3 + 0.0288252642822 s8 >= -0.0253319822943

_C32: s1 + s10 + s11 + s12 + s13 

Strategy Problem:
MAXIMIZE
628.8927248316759*s1 + -133.5260412463358*s10 + 4119.367836182006*s11 + -346.3852764200383*s12 + -153.33161821455724*s14 + -762.2203811101863*s15 + 558.3089589426927*s2 + -827.2472189608125*s3 + -666.6220579133224*s4 + 2013.136619659912*s5 + 633.3367506179602*s6 + 637.6884693675383*s7 + -2461.6028833355776*s8 + -3.88189339350356*s9 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: 0.0363294947211 s1 - 0.161497145956 s10 - 0.166747367147 s11
 - 0.0296437770261 s2 - 0.0627309003763 s5 + 0.00733588772706 s8
 >= -0.0661496825232

_C32: s1 + s10 

Strategy Problem:
MAXIMIZE
-854.2525290370977*s1 + 724.8199235198881*s10 + 346.3910557886775*s11 + -1726.8867076381175*s12 + 1748.2576102316016*s13 + 422.05530915597046*s14 + -767.0431023000165*s15 + 1058.3674125496982*s2 + -495.4204398447655*s3 + 181.04079519918014*s4 + -1136.4169503364974*s5 + -1543.646899277257*s6 + 678.1942431145144*s7 + 2091.399419002537*s8 + 1.7249026662102551*s9 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: 0.00453581817285 s1 - 0.0104195962176 s12 - 0.00484012592894 s14
 - 0.00870781773886 s2 - 0.0279291790291 s8 >= -0.0377023498695

_C32:

Strategy Problem:
MAXIMIZE
-417.8060698435438*s1 + 437.1067812536894*s10 + 864.255889072716*s11 + 115.27261571268173*s12 + -591.1709451969946*s13 + 213.34322751200307*s14 + 406.70799259177875*s15 + -257.90784557908046*s2 + 1562.6252441601973*s4 + -273.17558284975894*s5 + 291.9180574781749*s6 + 1003.1211923956434*s7 + -1347.2903735741784*s8 + -2556.513254187672*s9 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: - 0.0515005054493 s12 - 0.0215808993973 s14 - 0.0165136801029 s15
 - 0.0376500847228 s4 - 0.0410847332501 s8 + 0.0586966938189 s9
 >= -0.0488022225717

_C32: 

...

Strategy Problem:
MAXIMIZE
-1388.5711739278995*s1 + -1096.1966621892607*s10 + -100.04808999101078*s11 + 202.82819995670434*s12 + -776.3896265847652*s14 + 118.75680443747842*s15 + -65.32383645429945*s2 + 1062.2093355801844*s3 + -330.56377583387075*s4 + 241.18462462891694*s8 + -950.2698701008012*s9 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: - 0.0245209118058 s1 - 0.0423737604249 s11 + 0.00920677492422 s2
 - 0.0326566178616 s3 >= -0.0336407806967

_C32: s1 + s10 + s11 + s12 + s13 + s14 + s15 + s2 + s3 + s4 + s5 + s6 + s7
 + s8 + s9 = 15

VARIABLES
0 <= s1 <= 5 Int

Strategy Problem:
MAXIMIZE
234.08143569947424*s1 + 697.9161277882558*s10 + -2175.2446795656433*s11 + 1794.6400374609366*s12 + 100.95204777730396*s14 + -620.318984126221*s15 + -307.77932521197*s2 + -700.1544561840528*s4 + 681.8696056183944*s5 + 1052.9243753349692*s6 + -154.43489165362917*s7 + -561.5833725530101*s8 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: - 0.0254598111046 s1 - 0.032409292498 s12 - 0.0209011522761 s14
 + 0.0186187363283 s8 >= -0.0610800210456

_C32: s1 + s10 + s11 + s12 + s13 + s14 + s15 + s2 + s3 + s4 + s5 + s6 + s7
 + s8 + s9 = 15

VARIABLES


Strategy Problem:
MAXIMIZE
1278.8884600540368*s1 + 892.7905067542379*s10 + -102.43733723958212*s11 + 780.9590902229174*s12 + -291.7297650461187*s13 + 67.05196527308362*s14 + -191.4170491550649*s15 + 87.94954204453097*s2 + -475.80152797716437*s3 + 1320.240424937092*s4 + 716.6509417761481*s5 + 1311.1459406759277*s6 + -598.0931690431207*s8 + 541.0424020774444*s9 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: 0.0243533989471 s1 - 0.0392214685933 s12 - 0.0402353617401 s13
 - 0.019781854239 s14 - 0.00799587180553 s2 - 0.0288186519516 s4
 - 0.0155173218425 s5 - 0.02571141

Strategy Problem:
MAXIMIZE
-2541.036613051914*s1 + 844.9057673232201*s10 + -277.55768918459944*s11 + 1446.4155528554147*s12 + 499.9403598118515*s13 + 166.09236718808097*s14 + -174.68126895244131*s15 + 59.191663840672845*s2 + 749.171884683612*s3 + -1070.1732019596748*s4 + -272.17567177116507*s5 + -421.7785703850932*s6 + 493.1471096290734*s7 + -1179.3197173754852*s8 + -502.44840931511135*s9 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: 0.0211392349471 s1 + 0.0140253748558 s12 - 0.0360274047794 s13
 - 0.0174232003668 s14 - 0.0109071517378 s15 - 0.00736216337477 s2
 -

Strategy Problem:
MAXIMIZE
67.04186588097946*s1 + 553.7394088998626*s10 + 1920.3417489682106*s11 + -264.14067484892803*s12 + 618.4650277510009*s13 + 99.4583948793661*s14 + -802.1656803029728*s15 + 71.8165727422529*s2 + -122.31261443213225*s4 + -708.3656623633979*s5 + 43.628502461455355*s6 + 404.9922835208163*s7 + 1372.6386453721134*s8 + -503.3979128752035*s9 + 0.0
SUBJECT TO
_C1: s1 >= 0

_C2: s2 >= 0

_C3: s3 >= 0

_C4: s4 >= 0

_C5: s5 >= 0

_C6: s6 >= 0

_C7: s7 >= 0

_C8: s8 >= 0

_C9: s9 >= 0

_C10: s10 >= 0

_C11: s11 >= 0

_C12: s12 >= 0

_C13: s13 >= 0

_C14: s14 >= 0

_C15: s15 >= 0

_C16: s1 <= 5

_C17: s2 <= 5

_C18: s3 <= 5

_C19: s4 <= 5

_C20: s5 <= 5

_C21: s6 <= 5

_C22: s7 <= 5

_C23: s8 <= 5

_C24: s9 <= 5

_C25: s10 <= 5

_C26: s11 <= 5

_C27: s12 <= 5

_C28: s13 <= 5

_C29: s14 <= 5

_C30: s15 <= 5

_C31: - 0.0243050430504 s1 - 0.00883710501697 s2 - 0.000328994476763 s5
 - 0.00047604912827 s6 >= -0.035777385159

_C32: s1 + s10 + s11 + s12 + s13 + s14 + s15 + s2 + s3